<a href="https://colab.research.google.com/github/bamtak/masakhane/blob/master/yo_en_jw300_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Masakhane - Machine Translation for African Languages (Using JoeyNMT)

## Note before beginning:
### - The idea is that you should be able to make minimal changes to this in order to get SOME result for your own translation corpus. 

### - The tl;dr: Go to the **"TODO"** comments which will tell you what to update to get up and running

### - If you actually want to have a clue what you're doing, read the text and peek at the links

### - With 100 epochs, it should take around 7 hours to run in Google Colab

### - Once you've gotten a result for your language, please attach and email your notebook that generated it to masakhanetranslation@gmail.com

### - If you care enough and get a chance, doing a brief background on your language would be amazing. See examples in  [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

## Retrieve your data & make a parallel corpus

If you are wanting to use the JW300 data referenced on the Masakhane website or in our GitHub repo, you can use `opus-tools` to convert the data into a convenient format. `opus_read` from that package provides a convenient tool for reading the native aligned XML files and to convert them to TMX format. The tool can also be used to fetch relevant files from OPUS on the fly and to filter the data as necessary. [Read the documentation](https://pypi.org/project/opustools-pkg/) for more details.

Once you have your corpus files in TMX format (an xml structure which will include the sentences in your target language and your source language in a single file), we recommend reading them into a pandas dataframe. Thankfully, Jade wrote a silly `tmx2dataframe` package which converts your tmx file to a pandas dataframe. 

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "yo"
target_language = "en"
lc = False  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead! 
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt-$tag"
g_drive_path = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)
os.environ["gdrive_path"] = g_drive_path
models_path = '%s/models/%s%s_transformer'% (g_drive_path, source_language, target_language)
# model temporary directory for training
model_temp_dir = "/content/drive/My Drive/masakhane/model-temp"
# model permanent storage on the drive
!mkdir -p "$gdrive_path/models/${src}${tgt}_transformer/"

In [0]:
!echo $gdrive_path

/content/drive/My Drive/masakhane/yo-en-baseline


In [0]:
#TODO: Skip for retrain
# Install opus-tools
! pip install opustools-pkg 

     |████████████████████████████████| 81kB 9.5MB/s 


In [0]:
#TODO: Skip for retrain
# Downloading our corpus
! opus_read -d JW300 -s $src -t $tgt -wm moses -w jw300.$src jw300.$tgt -q

# extract the corpus file
! gunzip JW300_latest_xml_$src-$tgt.xml.gz


Alignment file /proj/nlpl/data/OPUS/JW300/latest/xml/en-yo.xml.gz not found. The following files are available for downloading:

   4 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en-yo.xml.gz
 263 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/en.zip
  58 MB https://object.pouta.csc.fi/OPUS-JW300/v1/xml/yo.zip

 325 MB Total size
./JW300_latest_xml_en-yo.xml.gz ... 100% of 4 MB
./JW300_latest_xml_en.zip ... 100% of 263 MB
./JW300_latest_xml_yo.zip ... 100% of 58 MB
gzip: JW300_latest_xml_yo-en.xml.gz: No such file or directory


In [0]:
# extract the corpus file
! gunzip JW300_latest_xml_$tgt-$src.xml.gz

In [0]:
#TODO: Skip for retrain
# Download the global test set.
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
# And the specific test set for this language pair.
os.environ["trg"] = target_language 
os.environ["src"] = source_language 

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$src.en 
! mv test.en-$src.en test.en
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$src.$src 
! mv test.en-$src.$src test.$src

--2020-04-07 20:43:18--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en’

test.en-any.en      100%[===================>] 271.28K  --.-KB/s    in 0.008s  

2020-04-07 20:43:19 (32.5 MB/s) - ‘test.en-any.en’ saved [277791/277791]

--2020-04-07 20:43:22--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-yo.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [0]:
#TODO: Skip for retrain
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

Loaded 3571 global test sentences to filter from the training/dev data.


In [0]:
#TODO: Skip for retrain
import pandas as pd

# TMX file to dataframe
source_file = 'jw300.' + source_language
target_file = 'jw300.' + target_language

source = []
target = []
skip_lines = []  # Collect the line numbers of the source portion to skip the same lines for the target portion.
with open(source_file) as f:
    for i, line in enumerate(f):
        # Skip sentences that are contained in the test set.
        if line.strip() not in en_test_sents:
            source.append(line.strip())
        else:
            skip_lines.append(i)             
with open(target_file) as f:
    for j, line in enumerate(f):
        # Only add to corpus if corresponding source was not skipped.
        if j not in skip_lines:
            target.append(line.strip())
    
print('Loaded data and skipped {}/{} lines since contained in test set.'.format(len(skip_lines), i))
    
df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])
# if you get TypeError: data argument can't be an iterator is because of your zip version run this below
#df = pd.DataFrame(list(zip(source, target)), columns=['source_sentence', 'target_sentence'])
df.head(3)

Loaded data and skipped 1025/474986 lines since contained in test set.


,source_sentence,target_sentence
0,Lílo Àkàbà — Ǹjẹ́ O Máa Ń Ṣe Àyẹ̀wò Wọ̀nyí Tó...,Using Ladders — Do You Make These Safety Checks ?
1,Látọwọ́ akọ̀ròyìn Jí !,By Awake !
2,ní Ireland,correspondent in Ireland


## Pre-processing and export

It is generally a good idea to remove duplicate translations and conflicting translations from the corpus. In practice, these public corpora include some number of these that need to be cleaned.

In addition we will split our data into dev/test/train and export to the filesystem.

In [0]:
#TODO: Skip for retrain
# drop duplicate translations
df_pp = df.drop_duplicates()

# drop conflicting translations
# (this is optional and something that you might want to comment out 
# depending on the size of your corpus)
df_pp.drop_duplicates(subset='source_sentence', inplace=True)
df_pp.drop_duplicates(subset='target_sentence', inplace=True)

# Shuffle the data to remove bias in dev set selection.
df_pp = df_pp.sample(frac=1, random_state=seed).reset_index(drop=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [0]:
#TODO: Skip for retrain
# Install fuzzy wuzzy to remove "almost duplicate" sentences in the
# test and training sets.
! pip install fuzzywuzzy
! pip install python-Levenshtein
import time
from fuzzywuzzy import process
import numpy as np

# reset the index of the training set after previous filtering
df_pp.reset_index(drop=False, inplace=True)

# Remove samples from the training data set if they "almost overlap" with the
# samples in the test set.

# Filtering function. Adjust pad to narrow down the candidate matches to
# within a certain length of characters of the given sample.
def fuzzfilter(sample, candidates, pad):
  candidates = [x for x in candidates if len(x) <= len(sample)+pad and len(x) >= len(sample)-pad] 
  if len(candidates) > 0:
    return process.extractOne(sample, candidates)[1]
  else:
    return np.nan

# NOTE - This might run slow depending on the size of your training set. We are
# printing some information to help you track how long it would take. 
scores = []
start_time = time.time()
for idx, row in df_pp.iterrows():
  scores.append(fuzzfilter(row['source_sentence'], list(en_test_sents), 5))
  if idx % 1000 == 0:
    hours, rem = divmod(time.time() - start_time, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds), "%0.2f percent complete" % (100.0*float(idx)/float(len(df_pp))))

# Filter out "almost overlapping samples"
df_pp['scores'] = scores
df_pp = df_pp[df_pp['scores'] < 95]

     |████████████████████████████████| 51kB 8.7MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144794 sha256=0233a2657a58078318fa67995ba97920d36a754bc5c5ae6bd1044938e414d250
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-Levenshtein
00:00:00.16 0.00 percent complete
00:00:23.21 0.24 percent complete
00:00:46.27 0.48 percent complete
00:01:09.18 0.71 percent complete
00:01:31.34 0.95 percent complete


00:01:54.26 1.19 percent complete
00:02:17.83 1.43 percent complete
00:02:40.41 1.66 percent complete
00:03:03.84 1.90 percent complete
00:03:26.18 2.14 percent complete
00:03:49.14 2.38 percent complete
00:04:11.64 2.61 percent complete
00:04:34.39 2.85 percent complete
00:04:58.78 3.09 percent complete
00:05:20.46 3.33 percent complete
00:05:43.36 3.56 percent complete
00:06:06.61 3.80 percent complete
00:06:28.24 4.04 percent complete
00:06:50.05 4.28 percent complete


00:07:14.21 4.51 percent complete
00:07:36.24 4.75 percent complete
00:07:58.82 4.99 percent complete
00:08:20.53 5.23 percent complete
00:08:42.12 5.46 percent complete
00:09:05.54 5.70 percent complete
00:09:27.72 5.94 percent complete
00:09:49.86 6.18 percent complete
00:10:14.16 6.41 percent complete
00:10:37.63 6.65 percent complete
00:10:58.94 6.89 percent complete
00:11:21.80 7.13 percent complete
00:11:45.94 7.36 percent complete
00:12:11.02 7.60 percent complete
00:12:34.00 7.84 percent complete
00:12:55.73 8.08 percent complete
00:13:18.24 8.31 percent complete
00:13:41.73 8.55 percent complete
00:14:04.32 8.79 percent complete
00:14:28.38 9.03 percent complete
00:14:50.08 9.27 percent complete
00:15:12.34 9.50 percent complete
00:15:36.21 9.74 percent complete
00:15:59.59 9.98 percent complete
00:16:21.11 10.22 percent complete
00:16:44.50 10.45 percent complete
00:17:07.51 10.69 percent complete
00:17:31.10 10.93 percent complete
00:17:53.86 11.17 percent complete
00:18:16.

00:19:00.18 11.88 percent complete
00:19:23.67 12.12 percent complete
00:19:46.78 12.35 percent complete
00:20:09.21 12.59 percent complete
00:20:31.55 12.83 percent complete
00:20:54.99 13.07 percent complete


00:21:16.35 13.30 percent complete
00:21:40.35 13.54 percent complete
00:22:02.52 13.78 percent complete
00:22:26.25 14.02 percent complete
00:22:48.93 14.25 percent complete
00:23:10.91 14.49 percent complete


...

02:02:06.36 80.77 percent complete
02:02:27.16 81.01 percent complete


02:02:49.16 81.25 percent complete


02:03:10.46 81.48 percent complete
02:03:30.79 81.72 percent complete
02:03:51.99 81.96 percent complete
02:04:12.28 82.20 percent complete
02:04:33.60 82.44 percent complete
02:04:54.65 82.67 percent complete
02:05:16.59 82.91 percent complete
02:05:38.05 83.15 percent complete
02:05:59.48 83.39 percent complete
02:06:20.02 83.62 percent complete
02:06:41.69 83.86 percent complete
02:07:02.11 84.10 percent complete
02:07:23.23 84.34 percent complete
02:07:44.85 84.57 percent complete
02:08:06.74 84.81 percent complete
02:08:27.44 85.05 percent complete
02:08:48.06 85.29 percent complete
02:09:09.08 85.52 percent complete
02:09:30.60 85.76 percent complete
02:09:51.78 86.00 percent complete
02:10:12.14 86.24 percent complete
02:10:32.79 86.47 percent complete
02:10:54.70 86.71 percent complete
02:11:15.84 86.95 percent complete
02:11:35.87 87.19 percent complete
02:11:56.10 87.42 percent complete
02:12:16.95 87.66 percent complete
02:12:38.57 87.90 percent complete
02:12:58.93 88.14 pe

02:15:04.41 89.56 percent complete
02:15:26.29 89.80 percent complete
02:15:47.92 90.04 percent complete
02:16:09.38 90.27 percent complete
02:16:31.90 90.51 percent complete
02:16:53.25 90.75 percent complete
02:17:14.60 90.99 percent complete
02:17:36.53 91.23 percent complete
02:17:56.41 91.46 percent complete
02:18:17.04 91.70 percent complete
02:18:38.28 91.94 percent complete
02:18:59.20 92.18 percent complete
02:19:20.81 92.41 percent complete
02:19:42.87 92.65 percent complete
02:20:03.77 92.89 percent complete
02:20:24.92 93.13 percent complete
02:20:45.84 93.36 percent complete
02:21:07.57 93.60 percent complete
02:21:28.39 93.84 percent complete
02:21:48.66 94.08 percent complete
02:22:09.81 94.31 percent complete
02:22:30.95 94.55 percent complete
02:22:51.24 94.79 percent complete
02:23:12.67 95.03 percent complete
02:23:33.71 95.26 percent complete
02:23:53.89 95.50 percent complete
02:24:15.22 95.74 percent complete
02:24:36.14 95.98 percent complete
02:24:57.43 96.21 pe

02:26:23.14 97.16 percent complete
02:26:44.48 97.40 percent complete
02:27:05.88 97.64 percent complete
02:27:26.90 97.88 percent complete
02:27:47.24 98.11 percent complete
02:28:07.91 98.35 percent complete
02:28:28.42 98.59 percent complete
02:28:50.20 98.83 percent complete
02:29:11.01 99.06 percent complete
02:29:31.39 99.30 percent complete
02:29:51.79 99.54 percent complete
02:30:12.57 99.78 percent complete


In [0]:
#TODO: Skip for retrain
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.
import csv

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_pp["source_sentence"] = df_pp["source_sentence"].str.lower()
    df_pp["target_sentence"] = df_pp["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_pp.tail(num_dev_patterns) # Herman: Error in original
stripped = df_pp.drop(df_pp.tail(num_dev_patterns).index)

with open("train."+source_language, "w") as src_file, open("train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open("dev."+source_language, "w") as src_file, open("dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

#stripped[["source_sentence"]].to_csv("train."+source_language, header=False, index=False)  # Herman: Added `header=False` everywhere
#stripped[["target_sentence"]].to_csv("train."+target_language, header=False, index=False)  # Julia: Problematic handling of quotation marks.

#dev[["source_sentence"]].to_csv("dev."+source_language, header=False, index=False)
#dev[["target_sentence"]].to_csv("dev."+target_language, header=False, index=False)

# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head train.*
! head dev.*

==> train.en <==
It identifies us as Jesus ’ followers and as imitators of Jehovah , the Source of love .
To live honestly is to lead a better life .
Exports : Oil , cocoa , coffee , cotton , wood , aluminum
Your reminders are what I am fond of . ”
After a meal , the pancreas responds to increases in the glucose content of the blood , releasing the proper amount of insulin
Jehovah invites people of all nations to draw close to him in prayer .
4 : 18 - 22 .
But when the Israelites were delivered from Egyptian bondage , the prophet Moses had Joseph’s bones taken along for burial in the Promised Land .
The sea was getting rougher , and the fight to stay afloat made me very tired .
Joyous and Thankful Despite Loss ( N .

==> train.yo <==
Òun ló ń jẹ́ káwọn èèyàn dá wa mọ̀ pé ọmọlẹ́yìn Jésù ni wá àti pé à ń fara wé Jèhófà , Ọlọ́run ìfẹ́ .
Sísọ bá a ṣe jẹ́ gan - an máa ń jẹ́ kí ìgbésí ayé èèyàn dára .
Ohun Àmúṣọrọ̀ : Epo rọ̀bì , kòkó , kọfí , òwú , igi àti tángaran
Àwọn ìránnilétí rẹ ni mo n



---


## Installation of JoeyNMT

JoeyNMT is a simple, minimalist NMT package which is useful for learning and teaching. Check out the documentation for JoeyNMT [here](https://joeynmt.readthedocs.io)  

In [3]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 2380 (delta 0), reused 0 (delta 0), pack-reused 2377
Receiving objects: 100% (2380/2380), 2.60 MiB | 13.71 MiB/s, done.
Resolving deltas: 100% (1670/1670), done.
Processing /content/joeynmt
     |████████████████████████████████| 61kB 2.2MB/s 
     |████████████████████████████████| 276kB 8.9MB/s 
     |████████████████████████████████| 307kB 15.4MB/s 
     |████████████████████████████████| 17.1MB 198kB/s 
     |████████████████████████████████| 215kB 54.7MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 747kB 54.8MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=73768 sha256=8f0158e4be23430ea152ac5f4ce11767926f7418820f2e8bc8242188ce72e051
  Stored in directory: /tmp/pip-ephem-wheel-cac

# Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [0]:
#TODO: Skip for retrain
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
from os import path
os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language

# Learn BPEs on the training data.
os.environ["data_path"] = path.join("joeynmt", "data", source_language + target_language) # Herman! 
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

# Apply BPE splits to the development and test data.
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
! mkdir -p $data_path
! cp train.* $data_path
! cp test.* $data_path
! cp dev.* $data_path
! cp bpe.codes.4000 $data_path
! ls $data_path

# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path "$gdrive_path/vocab.txt"

# Some output
! echo "BPE Xhosa Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 "$gdrive_path/vocab.txt"  # Herman

bpe.codes.4000	dev.en	     test.bpe.yo     test.yo	   train.en
dev.bpe.en	dev.yo	     test.en	     train.bpe.en  train.yo
dev.bpe.yo	test.bpe.en  test.en-any.en  train.bpe.yo
bpe.codes.4000	dev.en	test.bpe.en  test.en-any.en  train.bpe.yo
dev.bpe.en	dev.yo	test.bpe.yo  test.yo	     train.en
dev.bpe.yo	models	test.en      train.bpe.en    train.yo
BPE Xhosa Sentences
The large shi@@ eld of faith ( See par@@ ag@@ r@@ ap@@ h@@ s 12 - 14 )
The hel@@ me@@ t of sal@@ vation ( See par@@ ag@@ r@@ ap@@ h@@ s 15 - 18 )
I’@@ ve found that people respon@@ d well when they see that you are pas@@ sion@@ ate about the Bible and are doing your best to help them . ”
The s@@ word of the spirit ( See par@@ ag@@ r@@ ap@@ h@@ s 19 - 20 )
With Jehovah’s help , we can stand fir@@ m against him !
Combined BPE Vocab
ỵ
bítì
ẹ̀rún
Ísír@@
×
ạ@@
Íjí@@
ẽ@@
Gẹ̀ẹ́@@
ş


# Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [0]:
def get_last_checkpoint(directory):
  last_checkpoint = ''
  try:
    for filename in os.listdir(directory):
      if 'best' in filename and filename.endswith(".ckpt"):
        return filename
      if not 'best' in filename and filename.endswith(".ckpt"):
          if not last_checkpoint or int(filename.split('.')[0]) > int(last_checkpoint.split('.')[0]):
            last_checkpoint = filename
  except FileNotFoundError as e:
    print('Error Occur ', e)
  return last_checkpoint

In [7]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
# the content of te folder will be overwrite when you start trainin
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"
last_checkpoint = get_last_checkpoint(models_path)
print('Last checkpoint :',last_checkpoint)

Last checkpoint : best.ckpt


In [0]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s' % (source_language, target_language)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}/train.bpe"
    dev:   "{gdrive_path}/dev.bpe"
    test:  "{gdrive_path}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}/vocab.txt"
    trg_vocab: "{gdrive_path}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "{gdrive_path}/models/{name}_transformer/{last_checkpoint}" # uncommented to load a pre-trained model from last checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 3                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "{model_temp_dir}"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.2
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.3
""".format(name=name, gdrive_path=os.environ["gdrive_path"], source_language=source_language, target_language=target_language, model_temp_dir=model_temp_dir, last_checkpoint=last_checkpoint)
with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [9]:
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
!cd joeynmt; python3 -m joeynmt train configs/transformer_$src$tgt.yaml

2020-04-12 09:02:47,813 Hello! This is Joey-NMT.
2020-04-12 09:02:47.955382: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2020-04-12 09:02:49,269 Total params: 12188160
2020-04-12 09:02:49,271 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_

In [0]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
!cp -r "/content/drive/My Drive/masakhane/model-temp/"* "$gdrive_path/models/${src}${tgt}_transformer/"

In [11]:
# Output our validation accuracy
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"


Steps: 278000	Loss: 40294.09375	PPL: 4.63137	bleu: 30.09912	LR: 0.00003529	
Steps: 279000	Loss: 40277.57422	PPL: 4.62846	bleu: 30.26302	LR: 0.00003529	
Steps: 280000	Loss: 40215.38281	PPL: 4.61752	bleu: 30.27509	LR: 0.00003529	
Steps: 281000	Loss: 40243.10938	PPL: 4.62240	bleu: 30.13151	LR: 0.00002471	
Steps: 282000	Loss: 40242.69141	PPL: 4.62232	bleu: 30.14656	LR: 0.00002471	
Steps: 283000	Loss: 40270.61719	PPL: 4.62723	bleu: 30.16707	LR: 0.00002471	
Steps: 284000	Loss: 40262.34766	PPL: 4.62578	bleu: 30.14547	LR: 0.00002471	
Steps: 285000	Loss: 40268.48047	PPL: 4.62686	bleu: 30.09764	LR: 0.00002471	
Steps: 286000	Loss: 40247.50000	PPL: 4.62317	bleu: 30.22054	LR: 0.00002471	
Steps: 287000	Loss: 40190.99219	PPL: 4.61324	bleu: 30.37087	LR: 0.00001729	
Steps: 288000	Loss: 40156.46875	PPL: 4.60718	bleu: 30.30116	LR: 0.00001729	
Steps: 289000	Loss: 40172.62109	PPL: 4.61002	bleu: 30.37191	LR: 0.00001729	
Steps: 290000	Loss: 40156.46484	PPL: 4.60718	bleu: 30.32872	LR: 0.00001729	
Steps: 29100

In [12]:
# Test our model
! cd joeynmt; python3 -m joeynmt test "$gdrive_path/models/${src}${tgt}_transformer/config.yaml"


2020-04-12 09:36:10,951 Hello! This is Joey-NMT.
2020-04-12 09:36:40,192  dev bleu:  30.48 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-04-12 09:37:12,208 test bleu:  39.44 [Beam search decoding with beam size = 5 and alpha = 1.0]
